In [1]:
!pip install pandas openpyxl requests beautifulsoup4

# Extracción Informaci de Web wol.jw.org - Semanas Reunión

In [7]:
# ========================================
# SCRIPT: EXTRACTOR DE INFORMACIÓN WOL - GOOGLE COLAB
# ========================================
# INSTRUCCIONES: Solo necesitas cambiar las URLs de abajo por las semanas que quieres procesar
# Ejemplo de URL: "https://wol.jw.org/es/wol/meetings/r4/lp-s/2024/11/3"

URLS_A_PROCESAR = [
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/41",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/42",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/43",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/44"
]

# ========================================
# INSTALACIÓN DE DEPENDENCIAS (SOLO PARA COLAB)
# ========================================

try:
    import google.colab
    print("🔧 Instalando dependencias para Google Colab...")
    !pip install requests beautifulsoup4 pandas openpyxl -q
    print("✅ Dependencias instaladas correctamente")
except ImportError:
    print("ℹ️  Ejecutándose fuera de Google Colab - usando dependencias del sistema")

# ========================================
# CÓDIGO PRINCIPAL
# ========================================

import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

def extraer_informacion(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        info = {
            "semana": "",
            "libro": "",
            "cancion_inicial": "",
            "tesoros": "",
            "segunda_cancion": "",
            "tercera_cancion": "",
            "titulos_maestros": [],
            "nvc_titulo1": "",
            "nvc_titulo2": "",
            "nvc_titulo3": "",
            "info_lectura_biblia": "",
            "info_estudio_libro": ""
        }

        # 1. EXTRAER SEMANA
        semana_elem = soup.find('h1', id='p1')
        if semana_elem:
            info["semana"] = semana_elem.get_text(strip=True)

        # 2. EXTRAER LIBRO DE ESTUDIO
        libro_elem = soup.find('h2', id='p2')
        if libro_elem:
            strong_tags = libro_elem.find_all('strong')
            if strong_tags:
                info["libro"] = " ".join([s.get_text(strip=True) for s in strong_tags])
            else:
                info["libro"] = libro_elem.get_text(strip=True)

        # 3. EXTRAER CANCIONES
        cancion_elem = soup.find('h3', id='p3')
        if cancion_elem:
            cancion_text = cancion_elem.get_text(strip=True)
            match = re.search(r'Canción\s+(\d+)', cancion_text)
            if match:
                info["cancion_inicial"] = f"Canción {match.group(1)}"

        todas_h3 = soup.find_all('h3')
        canciones_encontradas = []
        cancion_inicial_texto = cancion_elem.get_text(strip=True) if cancion_elem else ""

        for h3 in todas_h3:
            texto = h3.get_text(strip=True)
            if "Canción" in texto and texto != cancion_inicial_texto:
                match = re.search(r'Canción\s+(\d+)', texto)
                if match:
                    cancion_formateada = f"Canción {match.group(1)}"
                    if cancion_formateada not in canciones_encontradas:
                        canciones_encontradas.append(cancion_formateada)

        if len(canciones_encontradas) >= 1:
            info["segunda_cancion"] = canciones_encontradas[0]
        if len(canciones_encontradas) >= 2:
            info["tercera_cancion"] = canciones_encontradas[1]

        # 4. EXTRAER TESOROS DE LA BIBLIA
        tesoros_elem = soup.find(['h5', 'h4', 'h3'], id='p5')
        if tesoros_elem:
            info["tesoros"] = tesoros_elem.get_text(strip=True)

        # 5. EXTRAER INFORMACIÓN DE LECTURA DE LA BIBLIA
        lectura_div = soup.find('div', id='tt28')
        if lectura_div:
            p_elem = lectura_div.find('p', id='p17')
            if p_elem:
                texto_html = str(p_elem).replace('<em>', '').replace('</em>', '')
                temp_soup = BeautifulSoup(texto_html, 'html.parser')
                texto_completo = temp_soup.get_text(separator=' ', strip=True)
                match = re.search(r'\(\d+\s+mins?\.\)\s*(.+)', texto_completo)
                if match:
                    info["info_lectura_biblia"] = match.group(1).strip()

        # 6. EXTRAER INFORMACIÓN DEL ESTUDIO DEL LIBRO
        estudio_biblico_encontrado = False
        for h3 in soup.find_all('h3'):
            if h3.find('strong') and "Estudio bíblico de la congregación" in h3.get_text(strip=True):
                siguiente_div = h3.find_next_sibling('div')
                if siguiente_div:
                    p_elem = siguiente_div.find('p')
                    if p_elem:
                        texto_html = str(p_elem).replace('<em>', '').replace('</em>', '')
                        temp_soup = BeautifulSoup(texto_html, 'html.parser')
                        texto_completo = temp_soup.get_text(separator=' ', strip=True)
                        match = re.search(r'\(\d+\s+mins?\.\)\s*(.+)', texto_completo)
                        if match:
                            info["info_estudio_libro"] = match.group(1).strip()
                            estudio_biblico_encontrado = True
                            break

        if not estudio_biblico_encontrado:
            for div in soup.find_all('div', class_=lambda x: x and 'du-margin-inlineStart' in str(x)):
                p_elem = div.find('p')
                if p_elem and '(30 mins.)' in p_elem.get_text():
                    texto_completo = p_elem.get_text(separator=' ', strip=True)
                    if any(libro in texto_completo.lower() for libro in ['lfb', 'lmd', 'lvs']):
                        texto_html = str(p_elem).replace('<em>', '').replace('</em>', '')
                        temp_soup = BeautifulSoup(texto_html, 'html.parser')
                        texto_completo = temp_soup.get_text(separator=' ', strip=True)
                        match = re.search(r'\(\d+\s+mins?\.\)\s*(.+)', texto_completo)
                        if match:
                            info["info_estudio_libro"] = match.group(1).strip()
                            break

        # 7. EXTRAER TÍTULOS DE SEAMOS MEJORES MAESTROS
        maestros_element = None
        vida_cristiana_element = None

        for h2 in soup.find_all('h2'):
            texto = h2.get_text(strip=True)
            if "SEAMOS MEJORES MAESTROS" in texto:
                maestros_element = h2
            elif "NUESTRA VIDA CRISTIANA" in texto:
                vida_cristiana_element = h2

        if maestros_element and vida_cristiana_element:
            maestros_id = maestros_element.get('id', '')
            vida_id = vida_cristiana_element.get('id', '')

            if maestros_id and vida_id:
                maestros_num = int(maestros_id.replace('p', '')) if maestros_id.replace('p', '').isdigit() else 0
                vida_num = int(vida_id.replace('p', '')) if vida_id.replace('p', '').isdigit() else 1000

                for h3 in soup.find_all('h3'):
                    h3_id = h3.get('id', '')
                    if h3_id and h3_id.replace('p', '').isdigit():
                        h3_num = int(h3_id.replace('p', ''))
                        if maestros_num < h3_num < vida_num and h3.find('strong'):
                            strong_text = h3.find('strong').get_text(strip=True)
                            if not strong_text.startswith("Canción"):
                                info["titulos_maestros"].append(strong_text)

        # 8. EXTRAER TÍTULOS DE NUESTRA VIDA CRISTIANA
        titulos_nvc = []
        estudio_biblico_texto = None

        if vida_cristiana_element:
            vida_id = vida_cristiana_element.get('id', '')
            if vida_id and vida_id.replace('p', '').isdigit():
                vida_num = int(vida_id.replace('p', ''))
                for h3 in soup.find_all('h3'):
                    h3_id = h3.get('id', '')
                    if h3_id and h3_id.replace('p', '').isdigit():
                        h3_num = int(h3_id.replace('p', ''))
                        if h3_num > vida_num and h3.find('strong'):
                            strong_text = h3.find('strong').get_text(strip=True)
                            if "Estudio bíblico de la congregación" in strong_text:
                                estudio_biblico_texto = strong_text
                                break
                            elif not strong_text.startswith("Canción"):
                                titulos_nvc.append(strong_text)

        if not estudio_biblico_texto:
            for h3 in soup.find_all('h3'):
                if h3.find('strong') and "Estudio bíblico de la congregación" in h3.get_text(strip=True):
                    estudio_biblico_texto = h3.find('strong').get_text(strip=True)
                    break

        info["nvc_titulo1"] = titulos_nvc[0] if len(titulos_nvc) > 0 else ""
        info["nvc_titulo2"] = titulos_nvc[1] if len(titulos_nvc) > 1 else ""
        info["nvc_titulo3"] = estudio_biblico_texto if estudio_biblico_texto else "Estudio bíblico de la congregación"

        return info

    except Exception as e:
        print(f"Error al extraer información de {url}: {str(e)}")
        return None

def procesar_urls_wol(urls):
    datos = []
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"info-reunion_{timestamp}.xlsx"

    print("🔍 Extrayendo información de las páginas WOL...")
    print("-" * 50)

    for i, url in enumerate(urls, 1):
        try:
            print(f"📄 ({i}/{len(urls)}) Procesando: {url}")
            info = extraer_informacion(url)

            if info is None:
                print("   ❌ No se pudo extraer información")
                continue

            datos_basicos = {
                "Semana": info["semana"],
                "Libro": info["libro"],
                "Canción Inicial": info["cancion_inicial"],
                "Tesoros de la Biblia": info["tesoros"],
                "Segunda Canción": info["segunda_cancion"],
                "Tercera Canción": info["tercera_cancion"]
            }

            for j, titulo in enumerate(info["titulos_maestros"], 1):
                if j <= 4:
                    datos_basicos[f"Maestros Título {j}"] = titulo

            for j in range(len(info["titulos_maestros"]) + 1, 5):
                datos_basicos[f"Maestros Título {j}"] = ""

            datos_basicos["NVC Título 1"] = info["nvc_titulo1"]
            datos_basicos["NVC Título 2"] = info["nvc_titulo2"]
            datos_basicos["NVC Título 3"] = info["nvc_titulo3"]
            datos_basicos["Info Lectura Biblia"] = info["info_lectura_biblia"]
            datos_basicos["Info Estudio Libro"] = info["info_estudio_libro"]

            datos.append(datos_basicos)
            print(f"   ✅ Semana extraída: {info['semana']}")
            print(f"      • Libro: {info['libro']}")
            print(f"      • Canciones: {info['cancion_inicial']} | {info['segunda_cancion']} | {info['tercera_cancion']}")
        except Exception as e:
            print(f"   ❌ Error al procesar: {str(e)}")

        time.sleep(1)

    print("-" * 50)

    if datos:
        try:
            df = pd.DataFrame(datos)
            print(f"💾 Creando archivo Excel: {nombre_archivo}")
            with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='Info-reunion', index=False)
            print(f"   ✅ Archivo creado exitosamente: {nombre_archivo}")
            return datos, nombre_archivo
        except Exception as e:
            print(f"   ❌ Error al crear el archivo Excel: {str(e)}")
            return None, None
    else:
        print("   ⚠️  No hay datos para guardar")
        return None, None

def descargar_archivo_colab(nombre_archivo):
    try:
        from google.colab import files
        print(f"📥 Iniciando descarga del archivo: {nombre_archivo}")
        files.download(nombre_archivo)
        print("✅ Descarga iniciada - revisa tus descargas")
    except ImportError:
        print("ℹ️  No estás en Google Colab - el archivo está guardado en el directorio actual")
    except Exception as e:
        print(f"❌ Error al descargar: {str(e)}")

# ========================================
# EJECUCIÓN AUTOMÁTICA
# ========================================
if __name__ == "__main__":
    print("="*60)
    print("🔄 EXTRACTOR DE INFORMACIÓN WOL - VMC (GOOGLE COLAB)")
    print("="*60)
    print(f"📋 Se procesarán {len(URLS_A_PROCESAR)} páginas...")
    print()

    datos_procesados, archivo_creado = procesar_urls_wol(URLS_A_PROCESAR)

    if datos_procesados and archivo_creado:
        print("\n" + "="*60)
        print("✅ INFORMACIÓN EXTRAÍDA CORRECTAMENTE")
        print("="*60)
        print(f"📊 Total de semanas procesadas: {len(datos_procesados)}")
        print(f"📁 Archivo creado: {archivo_creado}")
        print(f"📋 Hoja de datos: 'Info-reunión'")

        print("\n🔍 DESCARGA DEL ARCHIVO:")
        descargar_archivo_colab(archivo_creado)

        print("\n🎉 ¡EXTRACCIÓN COMPLETADA EXITOSAMENTE!")
    else:
        print("\n❌ No se pudo procesar ninguna URL o crear el archivo")

    print("\n" + "="*60)


🔧 Instalando dependencias para Google Colab...
✅ Dependencias instaladas correctamente
🔄 EXTRACTOR DE INFORMACIÓN WOL - VMC (GOOGLE COLAB)
📋 Se procesarán 4 páginas...

🔍 Extrayendo información de las páginas WOL...
--------------------------------------------------
📄 (1/4) Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/41
   ✅ Semana extraída: 6-12 DE OCTUBRE
      • Libro: ECLESIASTÉS 5, 6
      • Canciones: Canción 42 | Canción 160 | Canción 34
📄 (2/4) Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/42
   ✅ Semana extraída: 13-19 DE OCTUBRE
      • Libro: ECLESIASTÉS 7, 8
      • Canciones: Canción 39 | Canción 151 | Canción 124
📄 (3/4) Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/43
   ✅ Semana extraída: 20-26 DE OCTUBRE
      • Libro: ECLESIASTÉS 9, 10
      • Canciones: Canción 30 | Canción 47 | Canción 28
📄 (4/4) Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/44
   ✅ Semana extraída: 27 DE OCTUBRE A 2 DE NOVIEMBRE
      • Libr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga iniciada - revisa tus descargas

🎉 ¡EXTRACCIÓN COMPLETADA EXITOSAMENTE!



# Copiar de Info-reunion a Formato

In [ ]:
# ========================================
# SCRIPT AUTOMÁTICO - UN SOLO MAPEO
# Copiar de Info-reunion a Formato
# ========================================

import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import range_boundaries
import re
import os

from google.colab import drive
drive.mount('/content/drive')

# Configuración
NOMBRE_ARCHIVO = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/Programación VMC_Septiembre-2024-2025.xlsx'

HOJA_ORIGEN = 'Info-reunion'
HOJA_DESTINO = 'Formato'
ESPACIADO_FILAS = 8

# MAPEO ÚNICO (con comentarios para completar)
MAPEO = {
    'Semana': 'B2',
    'Libro': 'I2',
    'Canción Inicial': 'D3',
    'Tesoros de la Biblia': 'C5',
    'Segunda Canción': 'D14',
    'Tercera Canción': 'D19',
    'Maestros Título 1': 'C9',
    'Maestros Título 2': 'C0',
    'Maestros Título 3': 'C11',
    'Maestros Título 4': 'C12',
    'NVC Título 1': 'C15',
    'NVC Título 2': 'C16',
    'NVC Título 3': 'C17',
    'Info Lectura Biblia': 'H7',
    'Info Estudio Libro': 'J17'
}

def copiar_filas_con_celdas_combinadas(archivo_excel, hoja_origen, hoja_destino,
                                       mapeo_celdas, espaciado=8):
    """Copia todas las filas de Info-reunion a Formato sin pedir nada al usuario."""
    try:
        print("\n🚀 Iniciando copia automática...")

        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)
        print(f"   ✅ {len(df_origen)} filas encontradas en hoja origen")

        # Filas a copiar (todas)
        filas_a_copiar = df_origen

        wb = load_workbook(filename=archivo_excel)
        hoja_dest = wb[hoja_destino]

        def escribir_en_celda(hoja, celda_ref, valor):
            for rango_combinado in hoja.merged_cells.ranges:
                min_col, min_row, max_col, max_row = range_boundaries(str(rango_combinado))
                celda_col = hoja[celda_ref].column
                celda_row = hoja[celda_ref].row
                if min_col <= celda_col <= max_col and min_row <= celda_row <= max_row:
                    hoja.cell(row=min_row, column=min_col).value = valor
                    return
            hoja[celda_ref] = valor

        def extraer_coords(celda):
            match = re.match(r'([A-Z]+)(\d+)', celda.upper())
            if match:
                return match.group(1), int(match.group(2))
            return None, None

        columnas_validas = {col: celda for col, celda in mapeo_celdas.items()
                           if col in df_origen.columns and celda}

        contador_celdas = 0
        for idx, (_, fila) in enumerate(filas_a_copiar.iterrows()):
            desplazamiento = idx * (espaciado + 18)
            for columna, celda_base in columnas_validas.items():
                if pd.notna(fila[columna]):
                    letra, numero_base = extraer_coords(celda_base)
                    if letra and numero_base:
                        celda_destino = f"{letra}{numero_base + desplazamiento}"
                        escribir_en_celda(hoja_dest, celda_destino, fila[columna])
                        contador_celdas += 1

        wb.save(archivo_excel)
        wb.close()
        print(f"✅ {contador_celdas} celdas copiadas correctamente")
        print(f"💾 Archivo actualizado: {os.path.basename(archivo_excel)}")

    except Exception as e:
        print(f"❌ ERROR: {e}")
        import traceback
        traceback.print_exc()

# Punto de entrada automático
if __name__ == "__main__":
    copiar_filas_con_celdas_combinadas(NOMBRE_ARCHIVO, HOJA_ORIGEN, HOJA_DESTINO, MAPEO, ESPACIADO_FILAS)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🚀 Iniciando copia automática...
   ✅ 5 filas encontradas en hoja origen
✅ 60 celdas copiadas correctamente
💾 Archivo actualizado: Programación VMC_Septiembre-2024-2025.xlsx


# Copiar informacón de la reunión a Formato

In [ ]:
# ========================================
# CÓDIGO MEJORADO Y AMIGABLE
# Copiar filas de Excel a celdas específicas
# ========================================

# Se importan todas las librerías necesarias al inicio
import pandas as pd
from openpyxl import load_workbook
import re
import os
import sys

## 1. CONFIGURACIÓN PRINCIPAL (AQUÍ AJUSTAS TODO)
# =================================================================

# Define la ruta base y el nombre del archivo.
# El script ajustará la ruta automáticamente si está en Colab.
RUTA_DRIVE = 'JW/Super VMC/Programación-VMC/Programador_VMC'
NOMBRE_ARCHIVO = 'Programación VMC_Septiembre-2024-2025.xlsx'

# Nombres de las hojas que vas a usar
HOJA_ORIGEN = 'BD ASIG'
HOJA_DESTINO = 'Formato'

# Espacio (en número de filas) entre cada bloque de datos pegado
ESPACIADO_FILAS = 8

# Mapeo de las columnas de origen a las celdas de destino.
# Revisa que los nombres de las columnas ('SEMANA', 'PRESIDENCIA', etc.)
# coincidan EXACTAMENTE con los de tu archivo Excel.
MAPEO_CELDAS = {
    'SEMANA': 'B2',
    'PRESIDENCIA': 'O2',
    'ORACIÓN': 'O3',
    'TESOROS DE LA BIBLIA': 'O5',
    'BUSQUEMOS PERLAS ESCONDIDAS': 'O6',
    'LECTURA DE LA BIBLIA': 'O7',
    'SMM ASIG 1 ESTUD': 'L9',
    'SMM ASIG 1 ACOMP': 'O9',
    'SMM ASIG 2 ESTUD': 'L10',
    'SMM ASIG 2 ACOMP': 'O10',
    'SMM ASIG 3 ESTUD': 'L11',
    'SMM ASIG 3 ACOMP': 'O11',
    'SMM ASIG 4 ESTUD': 'L12',
    'SMM ASIG 4 ACOMP': 'O12',
    'NVC PARTE 1': 'O15',
    'NVC PARTE 2': 'O16',
    'ESTUDIO LIBRO': 'O17',
    'LECTOR LIBRO': 'O18',
    'ORACIÓN FINAL': 'O19',
    'Info Lectura Biblia': 'H7',
    'Info Estudio Libro': 'J17'
}


## 2. LÓGICA DEL SCRIPT (No es necesario modificar esta parte)
# =================================================================

def copiar_filas_celdas_especificas(archivo_excel, hoja_origen, hoja_destino, mapeo_celdas, filtro=None, num_filas=None, espaciado=8):
    """Copia datos de una hoja a otra según la configuración."""
    try:
        print(f"\n{'='*60}\n🚀 INICIANDO PROCESO DE COPIA\n{'='*60}")
        if not os.path.exists(archivo_excel):
            raise FileNotFoundError(f"El archivo {os.path.basename(archivo_excel)} no se encontró en la ruta especificada.")

        print(f"1️⃣ Leyendo la hoja '{hoja_origen}'...")
        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)
        print(f"   ✅ Se encontraron {len(df_origen)} filas.")

        df_filtrado = df_origen.copy()
        if filtro:
            columna, valor = filtro
            print(f"\n2️⃣ Aplicando filtro: {columna} = '{valor}'")
            if columna not in df_filtrado.columns:
                print(f"   ❌ ¡Error! La columna para filtrar '{columna}' no existe en la hoja de origen.")
                return
            df_filtrado = df_filtrado[df_filtrado[columna].astype(str).str.upper() == str(valor).upper()]
            if df_filtrado.empty:
                print(f"   ⚠️ No se encontraron filas que cumplan con el filtro.")
                return

        filas_a_copiar = df_filtrado
        if num_filas:
            print(f"\n3️⃣ Seleccionando las últimas {num_filas} filas...")
            filas_a_copiar = df_filtrado.tail(num_filas)

        print(f"\n✅ Se procesarán {len(filas_a_copiar)} filas.")
        if filas_a_copiar.empty:
            print("No hay filas para procesar. Proceso terminado.")
            return

        print("\n4️⃣ Abriendo archivo para escritura...")
        wb = load_workbook(filename=archivo_excel)
        if hoja_destino not in wb.sheetnames:
            print(f"   ❌ ¡Error! La hoja de destino '{hoja_destino}' no existe.")
            return
        hoja_dest = wb[hoja_destino]

        print("\n5️⃣ Copiando datos...")
        def extraer_coords(celda):
            match = re.match(r'([A-Z]+)(\d+)', celda.upper())
            return match.group(1), int(match.group(2))

        for idx, (_, fila) in enumerate(filas_a_copiar.iterrows()):
            desplazamiento = idx * (espaciado + 18)
            for columna, celda_base in mapeo_celdas.items():
                if columna in fila.index and pd.notna(fila[columna]):
                    letra, numero_base = extraer_coords(celda_base)
                    hoja_dest[f"{letra}{numero_base + desplazamiento}"] = fila[columna]

        print("\n6️⃣ Guardando cambios...")
        wb.save(archivo_excel)
        wb.close()
        print(f"\n🎉 ¡PROCESO COMPLETADO EXITOSAMENTE! 🎉")
        print(f"   Se han actualizado los datos en '{os.path.basename(archivo_excel)}'")

    except FileNotFoundError as e:
        print(f"\n❌ ERROR DE ARCHIVO: {e}")
    except Exception as e:
        print(f"\n❌ ERROR INESPERADO: {e}")
        import traceback
        traceback.print_exc()

## 3. BLOQUE DE EJECUCIÓN (INTERACTIVO)
# =================================================================

def ejecutar_proceso_interactivo():
    # --- Detección de entorno y configuración de ruta ---
    IN_COLAB = 'google.colab' in sys.modules
    archivo_excel = ""
    if IN_COLAB:
        print("📍 Entorno de Google Colab detectado.")
        try:
            from google.colab import drive
            drive.mount('/content/drive', force_remount=True)
            archivo_excel = os.path.join('/content/drive/MyDrive', RUTA_DRIVE, NOMBRE_ARCHIVO)
            print("✅ Google Drive conectado.")
        except Exception as e:
            print(f"❌ Error al conectar con Google Drive: {e}")
            return
    else:
        print("🖥️ Entorno local detectado.")
        archivo_excel = NOMBRE_ARCHIVO

    # --- Menú Interactivo ---
    while True:
        print(f"\n{'='*60}")
        print("MENU DE OPCIONES - ¿Qué deseas hacer?")
        print(f"{'='*60}")
        print("1. Filtrar por un mes específico (ej: AGOSTO, SEPTIEMBRE)")
        print("2. Copiar las últimas 'N' filas")
        print("3. Procesar TODAS las filas de la hoja de origen")
        print("4. Salir")

        opcion = input("\n👉 Elige una opción (1-4): ")

        if opcion == '1':
            mes = input("   Escribe el nombre del mes que quieres filtrar: ").upper()
            copiar_filas_celdas_especificas(
                archivo_excel, HOJA_ORIGEN, HOJA_DESTINO, MAPEO_CELDAS,
                filtro=('MES', mes), espaciado=ESPACIADO_FILAS
            )
            break
        elif opcion == '2':
            try:
                n_filas = int(input("   ¿Cuántas de las últimas filas quieres copiar?: "))
                copiar_filas_celdas_especificas(
                    archivo_excel, HOJA_ORIGEN, HOJA_DESTINO, MAPEO_CELDAS,
                    num_filas=n_filas, espaciado=ESPACIADO_FILAS
                )
            except ValueError:
                print("   ❌ Error: Por favor, introduce un número válido.")
            break
        elif opcion == '3':
            copiar_filas_celdas_especificas(
                archivo_excel, HOJA_ORIGEN, HOJA_DESTINO, MAPEO_CELDAS,
                espaciado=ESPACIADO_FILAS
            )
            break
        elif opcion == '4':
            print("\n👋 Proceso cancelado por el usuario. ¡Hasta luego!")
            break
        else:
            print("\n❌ Opción no válida. Por favor, elige un número del 1 al 4.")

# --- Ejecuta el proceso ---
ejecutar_proceso_interactivo()

📍 Entorno de Google Colab detectado.
Mounted at /content/drive
✅ Google Drive conectado.

MENU DE OPCIONES - ¿Qué deseas hacer?
1. Filtrar por un mes específico (ej: AGOSTO, SEPTIEMBRE)
2. Copiar las últimas 'N' filas
3. Procesar TODAS las filas de la hoja de origen
4. Salir

👉 Elige una opción (1-4): 1
   Escribe el nombre del mes que quieres filtrar: SEPTIEMBRE

🚀 INICIANDO PROCESO DE COPIA
1️⃣ Leyendo la hoja 'BD ASIG'...
   ✅ Se encontraron 59 filas.

2️⃣ Aplicando filtro: MES = 'SEPTIEMBRE'

✅ Se procesarán 5 filas.

4️⃣ Abriendo archivo para escritura...

5️⃣ Copiando datos...

6️⃣ Guardando cambios...

🎉 ¡PROCESO COMPLETADO EXITOSAMENTE! 🎉
   Se han actualizado los datos en 'Programación VMC_Septiembre-2024-2025.xlsx'


In [ ]:
!ls "/content/drive/MyDrive/Programación-VMC/"

ls: cannot access '/content/drive/MyDrive/Programación-VMC/': No such file or directory
